In [ ]:
import torch
from torch import nn
from torch.autograd import Variable

import losswise

from prettytable import PrettyTable
from tqdm import tqdm
import numpy as np
import os
import sys
import pickle
import random

from datasets import BurstDataset, ShuffledBatchSequentialSampler, FakeBurstDataset
from prep_dataset import BurstDatasetStandardizer
from models import Encoder, Decoder
from eval_functions import plot_autoencoding, autoencode, encode

import matplotlib.pyplot as plt

sys.path.append('../')
import utils
from readers.patient_info import PatientInfo

### Load datasets

In [ ]:
SAVE_DIR = 'saved_encs/config18/'

In [ ]:
params_dict = pickle.load(open(os.path.join(SAVE_DIR, 'params_dict.pkl')))
(train_dataset, dev_dataset, test_dataset) = pickle.load(open(os.path.join(SAVE_DIR, 'datasets.pkl')))
standardizer = pickle.load(open(os.path.join(SAVE_DIR, 'standardizer.pkl')))

### Load models

In [ ]:
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

In [ ]:
HIDDEN_SIZE = params_dict['hidden_size']
INPUT_SIZE = 1 # This CANNOT be changed! 
BIDIRECTIONAL = params_dict['bidirectional']
NUM_LAYERS = params_dict['num_layers']
EXTRA_INPUT_DIM = params_dict['extra_input_dim']

encoder = Encoder(INPUT_SIZE, HIDDEN_SIZE, bidirectional=BIDIRECTIONAL, num_layers=NUM_LAYERS)
decoder = Decoder(HIDDEN_SIZE, INPUT_SIZE, extra_input_dim=EXTRA_INPUT_DIM, encoder_bidirectional=BIDIRECTIONAL, 
                  num_layers=NUM_LAYERS)
if torch.cuda.is_available():
    encoder = encoder.cuda()
    decoder = decoder.cuda()

In [ ]:
NUM_EPOCHS = params_dict['num_epochs']
# 278 is the best epoch for config18
use_epoch = 278 
encoder.load_state_dict(torch.load(os.path.join(SAVE_DIR, 'epoch{}_enc.pkl'.format(use_epoch))))
decoder.load_state_dict(torch.load(os.path.join(SAVE_DIR, 'epoch{}_dec.pkl'.format(use_epoch))))
encoder.eval()
decoder.eval()

## Plot the autoencoding

In [ ]:
TRAIN_REVERSED = params_dict['train reversed']

In [ ]:
len(train_dataset), len(test_dataset)

In [ ]:
downsample_factor = params_dict['downsample_factor']
robust = params_dict['robust_scale']

In [ ]:
%matplotlib inline
dataset = test_dataset
for i in range(len(dataset)-15, len(dataset)):
    sample = dataset[i]
    try:
        # for updated datasets
        undownsampled = dataset.get_undownsampled_item(i, standardizer, robust)
    except AttributeError:
        print('old dataset')
        # for old datasets
        undownsampled = None
    mse = plot_autoencoding(sample, encoder, decoder, toss_encoder_output=False, 
                            reverse=TRAIN_REVERSED, undownsampled=undownsampled)